In [1]:
from quake.models.qsvm.genetic_v4 import get_kernel_entry, to_quantum
import numpy as np
import pandas as pd
from collections import OrderedDict
from qiskit_ibm_provider import IBMProvider
from qiskit_machine_learning.kernels import FidelityStatevectorKernel
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from qiskit.circuit import QuantumCircuit
from random import shuffle
from qiskit.primitives import BackendSampler
from qiskit import transpile

In [2]:
suffix = "2024_04_13 -15_21_47"
data_cv, data_labels = np.load("C:/Users/pc/Desktop/work/data_cv.npy"), np.load("C:/Users/pc/Desktop/work/data_labels.npy")
useful_ft = [0, 1,2,5,6, 8, 10, 12, 14, 15, 17]
data_cv = data_cv[:, useful_ft]

genes = pd.read_csv("../../Output_genetic/"+suffix+"/genes" +
                 suffix+".csv", header=None, index_col=False).to_numpy()
rand_idxs = pd.read_csv("../../Output_genetic/"+suffix+"/rand_indices" +
                 suffix+".csv", header=None, index_col=False).to_numpy()[0]
gate_dict = OrderedDict(
    [
        ("single_non_parametric", ["Id", "X", "SX"]),
        ("single_parametric", ["RZ"]),
        ("two_non_parametric", ["ECR"]),
        ("two_parametric", []),
    ]
)

IBMProvider.save_account(token='468dbb546acdc519d5999381a9d30fc849d558af57ca14fb9c9572ce2f15b387f7a16cca2fad386252168a5a63a55fecc4e75637e6b2e7da9beb83bab5ec463d', overwrite=True)
#ADD WITH SESSION    
provider = IBMProvider(instance='ibm-q-cern/infn/qcnphepgw')
backend = provider.get_backend('ibm_nazca')
coupling_map = backend.coupling_map

qsvm_connections_original = [
    [0,1,2,3],
    [4,15,22,21],
    [25,26,16,8],
    [13,12,11,10],
    [20, 33,39,38],
    [27,28,29,30],
    [32,36,51,50],
    [40,41,53,60],
    [49,48,47,46],
    # [71,58,57,56],
    [64,63,62,72],
    [70,69,68,67],
    [77,78,79,80],
    [82,83,84,85],
    [87,88,89,74],
    [95,94,90,75],
    [96,97,98,99],
    [119,118,110,100],
    [104,105,106,93],
    [116,115,114,113],
    [111,122,121,120],
    [126,125,124,123]
]

In [3]:
genes[20]

array([ 0,  1,  1,  6,  7,  3,  2,  1,  9,  4,  1,  0,  1,  4,  3,  0,  2,
        1,  9,  0,  2,  2,  0,  8,  9,  1,  1,  0,  7, 10,  1,  0,  1,  4,
        1,  2,  2,  1, 10,  8,  4,  1,  1,  8,  7,  0,  1,  0,  7,  1,  0,
        0,  1,  2,  5,  4,  1,  0,  9,  2,  3,  0,  0,  3, 10,  3,  2,  1,
        8,  2,  1,  1,  0,  5,  9,  4,  0,  1,  5,  9,  0,  1,  1,  2,  9,
        2,  0,  0,  5,  3,  4,  0,  1,  5,  8,  0,  0,  1,  2,  0,  0,  0,
        1,  0,  4,  4,  0,  0,  8,  4,  3,  2,  0,  2,  6,  4,  0,  0,  2,
        8,  1,  1,  0,  9, 10,  3,  1,  0,  3,  1,  3,  0,  1,  0,  7,  4,
        0,  0,  1,  8,  0,  2,  1,  6,  0,  3,  1,  0,  5,  7,  3,  1,  1,
       10,  2,  3,  2,  0,  8,  5,  4,  0,  0,  8,  2,  4,  0,  1, 10,  8,
        3,  2,  1,  6,  1,  2,  2,  1,  0,  9], dtype=int64)

In [4]:
fmap1, x1 = to_quantum(np.array(genes[14]), gate_dict=gate_dict, nb_features=11, gates_per_qubits=9, nb_qubits=4, param_shift=0, physical_qubits = qsvm_connections_original[rand_idxs[0]],coupling_map=coupling_map)
fmap2, x2 = to_quantum(np.array(genes[15]), gate_dict=gate_dict, nb_features=11, gates_per_qubits=9, nb_qubits=4, param_shift=0, physical_qubits = qsvm_connections_original[rand_idxs[1]],coupling_map=coupling_map)


[126, 125, 124, 123] for qubit 124 I did nothing
[126, 125, 124, 123] for qubit 124 I did nothing


In [5]:
def statevector_eval(fmap, x_idxs):
    qker = FidelityStatevectorKernel(feature_map=fmap)
    qker_matrix = qker.evaluate(x_vec=data_cv[:, x_idxs])


    clf = SVC(kernel="precomputed")

    param_grid = {"C": [0.01, 0.1, 1, 10, 100, 1000, 10000]}
    # param_grid = {"C": [1]}

    grid_search = GridSearchCV(clf, param_grid, cv=4, scoring="accuracy")
    grid_search.fit(qker_matrix, data_labels)
    best_clf = grid_search.best_estimator_
    accuracy_cv = cross_val_score(
        best_clf, qker_matrix, data_labels, cv=4, scoring="accuracy"
    )
    print(best_clf)
    print(accuracy_cv.mean())

In [6]:
quantum_circuit_list = [fmap1, fmap2]
x_idxss = [x1, x2]

In [7]:
nb_qubits = 4
nb_cbits = nb_qubits*len(quantum_circuit_list)
shuffle_index = list(range(len(quantum_circuit_list)))
shuffle(shuffle_index) 
cbits = [item for item in range(0, nb_cbits)]
flattened_qsvm_connections = [0,2,4,6,1,3,5,7]
qsvm_connections = [[0,2,4,6],
                    [1,3,5,7]]

In [8]:
combined_circuits = []
max_circuit_per_job = 300
counter = 0
for i in range(100):
    print(i)
    for j in range(i+1, data_cv.shape[0]):
        combined_circuit = QuantumCircuit(127, nb_cbits)
        for k, rand_idx in enumerate(shuffle_index):
            # bind correctly without specifying "x"
            bound_circuit = quantum_circuit_list[k].assign_parameters(data_cv[i, x_idxss[k]]).compose(
                quantum_circuit_list[k].assign_parameters(data_cv[j, x_idxss[k]]).inverse())
            # bound_circuit = transpile(bound_circuit, basis_gates=basis_gates, coupling_map= coupling_map_model)
            combined_circuit.compose(bound_circuit, qubits=[
                                    qsvm_connections[rand_idx][l] for l in range(nb_qubits)], inplace=True)
        combined_circuit.measure(flattened_qsvm_connections, cbits)
        
        if counter % max_circuit_per_job == 0:
            combined_circuit_batch = []
        # combined_circuit_batch.append(combined_circuit)
        combined_circuit_batch.append(
            combined_circuit)
        # combined_circuit_batch.append(
        #     combined_circuit)
        if counter % max_circuit_per_job == max_circuit_per_job - 1 or counter == (100**2 - 100)/2 - 1:            
            combined_circuits.append(combined_circuit_batch)
        counter += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [9]:
from qiskit_aer import Aer
simulator = Aer.get_backend('qasm_simulator')

job_results = []
print("Running job")
job_executions = [simulator.run(combined_circuits[i], shots=8000) for i in range(counter // max_circuit_per_job + 1)]
for i in range(counter // max_circuit_per_job + 1):
    job_results.append(job_executions[i].result())

Running job


In [10]:
generation_kernels = np.zeros((100, 100, 2))

n_jobs= len(combined_circuits)
big_counts = []
for i in range(n_jobs):
    print("getting counts from job", i)
    big_counts.append(job_results[i].get_counts())
counter = 0
for i in range(100):
    for j in range(i+1, data_cv.shape[0]):
        print("now doing", counter)
        n_job = counter // max_circuit_per_job
        counts = big_counts[n_job][counter % max_circuit_per_job]
        counter += 1
        for k, rand_idx in enumerate(shuffle_index):
            generation_kernels[i, j, k] = get_kernel_entry(cbits[nb_qubits*rand_idx:nb_qubits*(rand_idx+1)], counts, nb_qubits)
for k in range(2):
    generation_kernels[:, :, k] += generation_kernels[:,
                                                        :, k].T + np.eye(100)

getting counts from job 0
getting counts from job 1
getting counts from job 2
getting counts from job 3
getting counts from job 4
getting counts from job 5
getting counts from job 6
getting counts from job 7
getting counts from job 8
getting counts from job 9
getting counts from job 10
getting counts from job 11
getting counts from job 12
getting counts from job 13
getting counts from job 14
getting counts from job 15
getting counts from job 16
now doing 0
now doing 1
now doing 2
now doing 3
now doing 4
now doing 5
now doing 6
now doing 7
now doing 8
now doing 9
now doing 10
now doing 11
now doing 12
now doing 13
now doing 14
now doing 15
now doing 16
now doing 17
now doing 18
now doing 19
now doing 20
now doing 21
now doing 22
now doing 23
now doing 24
now doing 25
now doing 26
now doing 27
now doing 28
now doing 29
now doing 30
now doing 31
now doing 32
now doing 33
now doing 34
now doing 35
now doing 36
now doing 37
now doing 38
now doing 39
now doing 40
now doing 41
now doing 42
no

In [11]:
# from qiskit_aer import Aer
# simulator = Aer.get_backend('qasm_simulator')
# job_results = []
# for i in range(len(combined_circuits)):
#     print(i)
#     job_results.append(simulator.run(combined_circuits[i], shots=8000).result()) #.get_counts()

In [12]:
# generation_kernels = np.zeros((100, 100, 2))
# counter = 0
# for i in range(100):
#     for j in range(i+1, 100):

#         for k, rand_idx in enumerate(shuffle_index):
#             generation_kernels[i, j, k] = get_kernel_entry(cbits[nb_qubits*rand_idx:nb_qubits*(rand_idx+1)], job_results[counter].get_counts(), nb_qubits)
#         counter +=1
# # Symmetrizing and adding 1s to kernel diagonals
# for k in range(2):
#     generation_kernels[:, :, k] += generation_kernels[:,
#                                                         :, k].T + np.eye(100)

In [13]:
def qasm_eval(index):
    qker_matrix = generation_kernels[:,:,index]


    clf = SVC(kernel="precomputed")

    param_grid = {"C": [0.01, 0.1, 1, 10, 100, 1000, 10000]}
    grid_search = GridSearchCV(clf, param_grid, cv=4, scoring="accuracy")
    grid_search.fit(qker_matrix, data_labels)
    best_clf = grid_search.best_estimator_
    accuracy_cv = cross_val_score(
        best_clf, qker_matrix, data_labels, cv=4, scoring="accuracy"
    )
    print(best_clf)
    print(accuracy_cv.mean())

In [17]:
statevector_eval(fmap2, x2)

SVC(C=1, kernel='precomputed')
0.61


In [18]:
qasm_eval(1)

SVC(C=1, kernel='precomputed')
0.61
